# 2.1 ANALISI DELLA RETE: *NETWORK CHARACTERIZATION*

In questa sezione si analizzano le seguenti caratteristiche della rete creata precedentemente
1. [Informazioni di base](#info)
2. [Componenti connesse](#comps)
3. [Degree Distribution](#dd)
4. [Analisi dei path](#path)
5. [Analisi degli hubs](#hubs)
6. [Clustering Coefficient, densità](#cc)
7. [Centralità](#centralita)

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import warnings
import powerlaw
import seaborn as sns
import matplotlib.colors as mcolors
import pickle

warnings.filterwarnings('ignore')

from operator import itemgetter
from collections import Counter
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline 

In [3]:
df = pd.read_csv("../data_collection/data/df_grafo_pesato.csv", skipinitialspace=True, na_values='?', keep_default_na=True)   
del df['Unnamed: 0']
df

,source,id_source,target,id_target,weights
0,jwl2nd,4563241355,chanleycourttv,4730732546,1.000000
1,simplyslj,1518917881119059968,theemilydbaker,22594853,0.952055
2,jwl2nd,4563241355,courttv,1031946753510203392,0.924658
3,simplyslj,1518917881119059968,legalbytesmedia,1262475778631106560,0.914384
4,adrferg,1074757149748854792,courttv,1031946753510203392,0.633562
...,...,...,...,...,...
21708,intheone9,208950166,ThatUmbrella,901030183213445125,0.000000
21709,intheone9,208950166,dailymailuk,111556423,0.000000
21710,intheone9,208950166,elainebhoft,1380604861486665730,0.000000
21711,intheone9,208950166,girlsreallyrule,580312540,0.000000


Carico la rete reale (**g**) stabilendo la *source*, il *target* e l'attributo delle edges, nel nostro caso i pesi

In [4]:
g = nx.from_pandas_edgelist(df, source="source", target = "target", create_using=nx.MultiDiGraph()) #edge_attr = "weights"
nx.info(g) 

'MultiDiGraph with 18266 nodes and 21713 edges'

In [5]:
g2 = nx.Graph(g) # trasformo il grafo da multigrafo a grafo semplice indiretto
nx.info(g2) 

'Graph with 18266 nodes and 21697 edges'

## Informazioni di base <a id = 'info'> </a>

In [6]:
#funzione che stampa le informazioni di base delle reti (num nodi, edges, lmax ecc.)
def net_basic_info(g):
    nodes = g.number_of_nodes()
    edges = g.number_of_edges()
    directness = g.is_directed()
    self_loops = nx.number_of_selfloops(g)
    lMax = ((nodes) * (nodes - 1)) / 2
    AvgDegree = sum(dict(g.degree()).values())/float(len(g))

    print('- Numero Nodi:', nodes)
    print('- Numero Edges:', edges)
    print ("- Numero di self-loop: ", self_loops)
    print("- La rete e' diretta:",directness)
    print('- Numero massimo di collegamenti (L_max):', lMax)
    print('- Avarage Degree:', AvgDegree)


In [7]:
print("Caratteristiche Real World Netowrk (RETE REALE)\n ")
net_basic_info(g)


Caratteristiche Real World Netowrk (RETE REALE)
 
- Numero Nodi: 18266
- Numero Edges: 21713
- Numero di self-loop:  40
- La rete e' diretta: True
- Numero massimo di collegamenti (L_max): 166814245.0
- Avarage Degree: 2.377422533669112


## HUBS <a id = 'hubs'> </a>
Ricerco all'interno della rete gli **Hubs**

In [8]:
def find_hubs(g, t_degree):
    hubs = []
    noHubs = []
    for i in g: #scorro i nodi in g
        if g.degree(i) >=  t_degree: 
            #se il grado di quella rete è superiorie al grado soglia (t_degree) lo aggiunto alla lista degli hubs
            hubs.append(i)
        else: # altrimenti li aggiungo alla lista degli altri nodi
            noHubs.append(i)
    #infine, stampo il num di hubs identificati    
    print("Numero di hubs nella rete: ", len(hubs), '\n')
    
    return hubs, noHubs

Estraggo gli hubs con la funzione find_hub e ricerco l'hub maggiore (quello con grado più alto)

In [9]:
print('-----------------\n HUBS DELLA RETE \n-----------------\n')

hubs, noHubs = find_hubs(g, 60)  #impostando come soglia un grado pari a 60 ottendo 40 hubs

largest_hub, degree = sorted(g.degree(), key=itemgetter(1))[-1] #ricerco l'hub maggiore
print('Hub maggiore:', largest_hub , 'con', degree, 'collegamenti\n')
for i in hubs:
    sorted(hubs)
    print('*', i, ' - grado:', g.degree(i)) 


-----------------
 HUBS DELLA RETE 
-----------------

Numero di hubs nella rete:  40 

Hub maggiore: deppheardtruth con 548 collegamenti

* theemilydbaker  - grado: 370
* courttv  - grado: 205
* legalbytesmedia  - grado: 183
* youtube  - grado: 499
* queenofcodebass  - grado: 66
* thatumbrella  - grado: 294
* rekietalaw  - grado: 130
* GellertDepp  - grado: 145
* TheEmilyDBaker  - grado: 107
* realamberheard  - grado: 413
* deppheardtruth  - grado: 548
* tmz  - grado: 248
* ThatUmbrella  - grado: 172
* kyduiguy  - grado: 223
* lawcrimenetwork  - grado: 91
* DrProudman  - grado: 107
* reenie62  - grado: 133
* mensurvivetoo  - grado: 69
* ianrunkle  - grado: 173
* KyDUIGuy  - grado: 85
* ThatBrianFella  - grado: 78
* robert__leonard  - grado: 278
* elonmusk  - grado: 262
* jamesfromcourt  - grado: 112
* thirdgirlll  - grado: 72
* voguemagazine  - grado: 115
* andysignore  - grado: 108
* CourtTV  - grado: 117
* eve_barlow  - grado: 118
* lumberlaw  - grado: 107
* niermanjoe  - grado: 61


### EGO NETWORK: rete in cui al centro di trova il nodo con più 

---

## Componenti Connesse <a id = 'comps'> </a>

In [10]:
#per calcolare le componenti connesse è necessario trasformare la rete  da diretta a indiretta
comps_g = list(nx.connected_components(g.to_undirected()))  
print ("- Componenti connesse RETE REALE:", len(comps_g))

big_comp_g = len(comps_g[0])
print ('- Componente maaggiore:', big_comp_g)


- Componenti connesse RETE REALE: 2412
- Componente maaggiore: 11986


La rete con più componenti connesse è la rete random (ER)

### Componenti debolmente e fortemente connesse

In [11]:
scc_g = nx.number_strongly_connected_components(g)
wcc_g = nx.number_weakly_connected_components(g)

In [12]:
print ("- Componenti FORTEMENTE CONNESSE rete reale:", scc_g)
print ("- Componenti DEBOLMENTE CONNESSE rete reale:", wcc_g)

- Componenti FORTEMENTE CONNESSE rete reale: 18254
- Componenti DEBOLMENTE CONNESSE rete reale: 2412


****

*****

## Clustering Coefficient e Densità <a id = 'comps'> </a>

In [15]:
#passo g2 perché per calcolare il clustering coeff è necessario avere un grafo semplice e non un multigrafo
cc = nx.clustering(g2) 
ordine_cc_g = [(k, v) for k, v in sorted(cc.items(), key=lambda item: -item[1])]
ordine_cc_g[0:10]

[('jacxpearl', 1.0),
 ('goblinqchesh', 1.0),
 ('ihaveadream0101', 1.0),
 ('KevinRashidi', 1.0),
 ('courttvfans', 1.0),
 ('chrisablesfool', 1.0),
 ('robbloodysaul', 1.0),
 ('amypolacko', 1.0),
 ('ssolorio77', 1.0),
 ('blaw', 1.0)]

In [16]:
#passo g2 perché per calcolare il numero di triangoli è necessario avere un grafo semplice e non un multigrafo
triangoli = nx.triangles(g2)
ordine_triangoli_g = [(k, v) for k, v in sorted(triangoli.items(), key=lambda item: -item[1])]
ordine_triangoli_g[0:10]

[('deppheardtruth', 116),
 ('lawyerschiff', 60),
 ('courttv', 59),
 ('andysignore', 59),
 ('ianrunkle', 45),
 ('laurabockov', 42),
 ('thatbrianfella', 40),
 ('youtube', 35),
 ('chanleycourttv', 30),
 ('ashleycourttv', 29)]

In [17]:
print("AVG CLUSTERING COFFICIENT E DENSITA RETE REALE")
print ("- Densita':", nx.density(g),
       "\n- Average Clustering Coeff: ", nx.average_clustering(g2)) #passo all'avg clusteing g2, ossia il grafo semplice 

AVG CLUSTERING COFFICIENT E DENSITA RETE REALE
- Densita': 6.508137239718347e-05 
- Average Clustering Coeff:  0.008784776655445555


In [18]:
hist_g = nx.degree_histogram(g)

fig = plt.figure(figsize=(10, 8)) 
plt.plot(range(0, len(hist_g)), hist_g, "o", color = '#fb9a99')
plt.title("Degree Distribution nella rete reale", fontsize = 18)
plt.xlabel("Degree", fontsize = 14)
plt.ylabel("Nodi", fontsize = 14)
plt.loglog()
plt.savefig('plots/degree_dist_rete_reale.png')

plt.show()

**CCDF** (complementary cumulative distribution function/funzione di distribuzione cumulativa complementare): un metodo di calcolo statistico che misura la potenza nel tempo(?)


**CDF** (comulative distribuzion fraction) [https://en.wikipedia.org/wiki/Cumulative_distribution_function]

In [19]:
def plot_dist(G):
    M = nx.to_scipy_sparse_matrix(G)
    xmin = min([d[1] for d in G.degree()])
    indegrees = M.sum(0).A[0]
    degree = np.bincount(indegrees)
    fit = powerlaw.Fit(np.array(degree)+1, fit_method='KS')#, xmin=xmin, xmax=max(degree)-xmin,discrete=True)
    #plt.title('Distribution', fontsize = 20)
    
    fig = plt.figure(figsize=(16, 6)) 
    
    
    """ Plot Distribution """
    plt.subplot(1, 3, 1)
    plt.plot(range(len(degree)),degree,'b.', hist_g, "o", color = 'g')   
    plt.loglog()
    plt.xlim((min(degree), max(degree)))
    plt.title('Degree Distibution', fontsize = 16)
    plt.xlabel('Degree', fontsize = 14)
    plt.ylabel('P(k)', fontsize = 14)


    """ Plot CDF """
    plt.subplot(1, 3, 2)
    fit.plot_cdf(color = 'g' )
    plt.title("Comulative Degree Fraction", fontsize = 16)
    plt.xlabel("Degree", fontsize = 14)
    plt.ylabel('CDF', fontsize = 14)

    """ Plot CCDF """
    plt.subplot(1, 3, 3)
    fit.power_law.plot_pdf(color='r',linestyle='-',label='fit ccdf')
    fit.plot_ccdf(color = 'g')
    plt.title("Complementary Comulative Degree Fraction", fontsize = 16)
    plt.ylabel('CCDF', fontsize = 14)
    plt.xlabel('Degree', fontsize = 14)
    plt.tight_layout()
    plt.show()
    print(f'alpha={fit.power_law.alpha+1} sigma={fit.power_law.sigma}')

Calculating best minimal value for power law fit


alpha=2.555352631600206 sigma=0.10891361174825213


In [20]:
plot_dist(g)

----

## Path Analysis <a id = 'path'> </a>

In [21]:
#creo dei samples per ciascuna rete così da ridurre i tempi die esecuzione 

In [22]:
print("Average shortest path (g):", nx.average_shortest_path_length(g.subgraph(comps_g[0])))
print("Average shortest path weighted (g):", nx.average_shortest_path_length(g.subgraph(comps_g[0]), weight='weight'))

Average shortest path (g): 0.0002976494409657881
Average shortest path weighted (g): 0.0002976494409657881


----

## Centralità <a id = 'centralita'> </a>

Di seguito verranno calcolare sulla rete le le seguenti **misure di centralità** e ordinate in base all'importante (solitamente i 10 nodi con centralità maggiore)
1. [Degree Centrality](#dc)
2. [Closeness Centrality](#clc)
3. [Betweenness Centrality](#bc)
4. [Harmonic Centrality](#hc)
5. [Eigenvector Centrality](#ec) 
6. [Page Rank](#pr)


 ### Degree Centrality <a id="dc"></a>

Creo un dizionario con i gradi dei nodi del dizionario e stampo i 10 nodi con grado maggiore

In [23]:
#Creo un dizionario con i gradi
degrees_g = dict(g.degree())

In [24]:
ordine_gradi_g = [(k, v) for k, v in sorted(degrees_g.items(), key=lambda item: -item[1])]
ordine_gradi_g[0:10]

[('deppheardtruth', 548),
 ('youtube', 499),
 ('realamberheard', 413),
 ('theemilydbaker', 370),
 ('thatumbrella', 294),
 ('robert__leonard', 278),
 ('elonmusk', 262),
 ('tmz', 248),
 ('kyduiguy', 223),
 ('courttv', 205)]

### Closeness Centrality  <a id="clc"></a>

In [25]:
cc_g = nx.closeness_centrality(g)

In [26]:
ordine_cc_g = [(k, v) for k, v in sorted(cc_g.items(), key=lambda item: -item[1])]
ordine_cc_g[0:10]

[('youtube', 0.029215545229780107),
 ('elonmusk', 0.02214318464219611),
 ('realamberheard', 0.021937620995456576),
 ('theemilydbaker', 0.019899932950156476),
 ('thatumbrella', 0.016826254121627787),
 ('tmz', 0.015306515502561244),
 ('twitter', 0.013246927016217876),
 ('kyduiguy', 0.01239196457915949),
 ('courttv', 0.011312697240314242),
 ('anoushasakoui', 0.010891105484077014)]

### Betweenness Centrality <a id="bc"></a>

In [27]:
bc_g = nx.closeness_centrality(g)

In [28]:
ordine_bc_g = [(k, v) for k, v in sorted(bc_g.items(), key=lambda item: -item[1])]
ordine_bc_g[0:10]

[('youtube', 0.029215545229780107),
 ('elonmusk', 0.02214318464219611),
 ('realamberheard', 0.021937620995456576),
 ('theemilydbaker', 0.019899932950156476),
 ('thatumbrella', 0.016826254121627787),
 ('tmz', 0.015306515502561244),
 ('twitter', 0.013246927016217876),
 ('kyduiguy', 0.01239196457915949),
 ('courttv', 0.011312697240314242),
 ('anoushasakoui', 0.010891105484077014)]

### Harmonic Centrality <a id="hc"></a>

In [29]:
hc_g = nx.harmonic_centrality(g)

In [30]:
ordine_hc_g = [(k, v) for k, v in sorted(hc_g.items(), key=lambda item: -item[1])]
ordine_hc_g[0:10]

[('youtube', 653.6166666666666),
 ('elonmusk', 470.01666666666654),
 ('realamberheard', 439.0333333333333),
 ('theemilydbaker', 398.70000000000005),
 ('thatumbrella', 350.28333333333336),
 ('tmz', 326.40000000000003),
 ('twitter', 265.13333333333344),
 ('kyduiguy', 248.95000000000002),
 ('courttv', 218.83333333333331),
 ('twitterspaces', 206.13333333333335)]

## Eigenvector Centrality <a id="ec"></a>

In [31]:
evc_g = nx.eigenvector_centrality(g2)

In [32]:
ordine_evc_g = [(k, v) for k, v in sorted(evc_g.items(), key=lambda item: -item[1])]
ordine_evc_g[0:10]

[('deppheardtruth', 0.43145903095568294),
 ('theemilydbaker', 0.25008747713715995),
 ('youtube', 0.19950251392186633),
 ('thatumbrella', 0.19023553647039124),
 ('ianrunkle', 0.16351526973353717),
 ('kyduiguy', 0.15077600615282097),
 ('realamberheard', 0.14907270337806114),
 ('legalbytesmedia', 0.1282491942492024),
 ('lumberlaw', 0.11295300716421157),
 ('tmz', 0.09982800674732849)]

### Page Rank <a id="pr"></a>

In [33]:
pr_g = nx.pagerank(g)

In [34]:
ordine_pr_g = [(k, v) for k, v in sorted(pr_g.items(), key=lambda item: -item[1])]
ordine_pr_g[0:10]

[('youtube', 0.013702530783329336),
 ('realamberheard', 0.008800819490335761),
 ('theemilydbaker', 0.0070191844212254976),
 ('lawcrimenetwork', 0.006281951158623227),
 ('elonmusk', 0.006154076251624446),
 ('tmz', 0.004551395950647417),
 ('thatumbrella', 0.0044636433269262905),
 ('courttv', 0.00387811948482185),
 ('ashleycourttv', 0.0034249601617000235),
 ('nypost', 0.003151876601070656)]